<a href="https://colab.research.google.com/github/houyingshi/tensorflow_basic_development/blob/master/tf_train_saver_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""
@author: Yingshi Hou
"""

import tensorflow as tf
import numpy as np
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

train_X = np.linspace(-1, 1, 100)
train_Y = 2 * train_X + np.random.randn(*train_X.shape) * 0.3  # y=2x，但是加入了噪声


X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
W = tf.Variable(tf.random_normal([1]), name="weight")
b = tf.Variable(tf.zeros([1]), name="bias")

z = tf.multiply(X, W) + b

cost = tf.reduce_mean(tf.square(Y - z))
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)  # Gradient descent

init = tf.global_variables_initializer()
training_epochs = 20
display_step = 2

saver = tf.train.Saver(max_to_keep = 1)
saver_step = 0
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X: x, Y: y})

        if epoch % display_step == 0:
            loss = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
            print("Epoch:", epoch + 1, "cost=", loss, "W=", sess.run(W), "b=", sess.run(b))
            saver.save(sess, 'liner_regression/model', global_step =epoch)
            saver_step=epoch

    print(" Finished!")
    print("cost=", sess.run(cost, feed_dict={X: train_X, Y: train_Y}), "W=", sess.run(W), "b=", sess.run(b))



    print("x=0.2，z=", sess.run(z, feed_dict={X: 0.2}))

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, 'liner_regression/model-' + str(saver_step))
    print("x=0.2，z=", sess.run(z, feed_dict={X: 0.2}))

print_tensors_in_checkpoint_file('liner_regression/model-' + str(saver_step), None, True)